In [1]:
import time 
import requests
from bs4 import BeautifulSoup 
import pandas as pd

**Contract Data**

In [2]:
resp = requests.get("https://overthecap.com/contracts/")
soup = BeautifulSoup(resp.content, "html.parser") 

tables = soup.find_all("table")
rows = []

data = tables[0].find_all("tr")[1:]
rows = []
for i in range(len(data)):
    player_info = data[i].find_all("td")
    name = player_info[0].text
    position = player_info[1].text
    team = player_info[2].text
    total_val = player_info[3].text
    avg_per_year = player_info[4].text
    total_guranteed = player_info[5].text
    avg_guranteed_per_year = player_info[6].text
    percent_guranteed = player_info[7].text
    
    rows.append({
        "Name" : name,
        "Position" : position,
        "Team" : team,
        "Total Value" : total_val,
        "Average Per Year" : avg_per_year,
        "Total Guranteed" : total_guranteed,
        "Average Guranteed Per Year" : avg_guranteed_per_year,
        "Percent Guranteed" : percent_guranteed
    })

In [3]:
df = pd.DataFrame(rows)
df.head()

,Average Guranteed Per Year,Average Per Year,Name,Percent Guranteed,Position,Team,Total Guranteed,Total Value
0,"$17,500,000","$35,000,000",Russell Wilson,50.0%,QB,Seahawks,"$70,000,000","$140,000,000"
1,"$18,750,000","$34,000,000",Ben Roethlisberger,55.1%,QB,Steelers,"$37,500,000","$68,000,000"
2,"$14,260,671","$33,500,000",Jared Goff,42.6%,QB,Rams,"$57,042,682","$134,000,000"
3,"$19,675,000","$33,500,000",Aaron Rodgers,58.7%,QB,Packers,"$78,700,000","$134,000,000"
4,"$30,500,000","$33,000,000",Kirk Cousins,92.4%,QB,Vikings,"$61,000,000","$66,000,000"


**QB Data**

In [6]:
df_temp_list = []
for i in [1,2]:
    # i = page number
    resp = requests.get("http://www.nfl.com/stats/categorystats?tabSeq=1&season=2019&seasonType=REG&experience=&Submit=Go&archive=false&conference=null&d-447263-p={}&statisticPositionCategory=QUARTERBACK&qualified=true".format(i))
    soup = BeautifulSoup(resp.content, "html.parser") 

    tables = soup.find_all("table")
    rows = []
    for player in tables[0].find_all("tr")[1:]:
        cell = player.find_all("td")
        name = cell[1].text.strip("\n")
        team = cell[2].text.strip("\n")
        position = cell[3].text
        attempts = int(cell[5].text.strip())
        if cell[6].text.strip() == "--":
            completion_percent = 0.0
        else:
            completion_percent = float(cell[6].text.strip())
        yards = int(cell[8].text.strip().replace(",", ""))
        yards_per_game = float(cell[10].text.strip())
        tds = int(cell[11].text.strip())
        ints = int(cell[12].text.strip())
        sacks = int(cell[18].text.strip())
        qbr = float(cell[19].text.strip())

        rows.append({
            "Name" : name,
            "Team" : team,
            "Position" : position,
            "Attempts" : attempts,
            "Completion Percentage" : completion_percent,
            "Yards" : yards,
            "Yards Per Game" : yards_per_game,
            "TDs" : tds,
            "Ints": ints,
            "Sacks" : sacks,
            "QBR" : qbr
        })
    time.sleep(.25)
    
    temp_df = pd.DataFrame(rows)
    df_temp_list.append(temp_df)

In [7]:
qb = pd.concat(df_temp_list, ignore_index=True) 
qb.head()

,Attempts,Completion Percentage,Ints,Name,Position,QBR,Sacks,TDs,Team,Yards,Yards Per Game
0,626,60.7,30,Jameis Winston,QB,84.3,47,33,TB,5109,319.3
1,596,65.1,11,Dak Prescott,QB,99.7,23,30,DAL,4902,306.4
2,626,62.9,16,Jared Goff,QB,86.5,22,22,LA,4638,289.9
3,591,66.0,20,Philip Rivers,QB,88.5,34,23,LAC,4615,288.4
4,616,66.2,14,Matt Ryan,QB,92.1,48,26,ATL,4466,297.7


**QB plus Salary**

In [8]:
qb_sal = pd.merge(qb, df, on=["Name", "Position"])
qb_sal.head()

,Attempts,Completion Percentage,Ints,Name,Position,QBR,Sacks,TDs,Team_x,Yards,Yards Per Game,Average Guranteed Per Year,Average Per Year,Percent Guranteed,Team_y,Total Guranteed,Total Value
0,626,60.7,30,Jameis Winston,QB,84.3,47,33,TB,5109,319.3,"$6,337,819","$6,337,819",100.0%,Buccaneers,"$25,351,277","$25,351,277"
1,596,65.1,11,Dak Prescott,QB,99.7,23,30,DAL,4902,306.4,$0,"$31,509,000",0.0%,Cowboys,$0,"$31,509,000"
2,626,62.9,16,Jared Goff,QB,86.5,22,22,LA,4638,289.9,"$14,260,671","$33,500,000",42.6%,Rams,"$57,042,682","$134,000,000"
3,591,66.0,20,Philip Rivers,QB,88.5,34,23,LAC,4615,288.4,"$25,000,000","$25,000,000",100.0%,Colts,"$25,000,000","$25,000,000"
4,616,66.2,14,Matt Ryan,QB,92.1,48,26,ATL,4466,297.7,"$18,900,000","$30,000,000",63.0%,Falcons,"$94,500,000","$150,000,000"


In [9]:
qb_sal.to_csv("NFL_salary_QB_data2020", header=True, index=False)

**RB Data**

In [10]:
df_temp_list = []
for i in [1,2,3]:
    # i = page number
    resp = requests.get("http://www.nfl.com/stats/categorystats?tabSeq=1&season=2019&seasonType=REG&experience=&Submit=Go&archive=false&d-447263-p={}&conference=null&statisticPositionCategory=RUNNING_BACK&qualified=true".format(i))
    soup = BeautifulSoup(resp.content, "html.parser") 

    tables = soup.find_all("table")
    rows = []
    for player in tables[0].find_all("tr")[1:]:
        cell = player.find_all("td")
        name = cell[1].text.strip("\n")
        team = cell[2].text.strip("\n")
        position = cell[3].text
        attempts = int(cell[4].text.strip())
        attempts_per_game = float(cell[5].text.strip())
        yards = int(cell[6].text.strip().replace(",", ""))
        avg_per_carry = float(cell[7].text.strip())
        yards_per_game = float(cell[8].text.strip())
        TDs = int(cell[9].text.strip())
        twenty = int(cell[13].text.strip())
        forty = int(cell[14].text.strip())
        fum = int(cell[15].text.strip())

        rows.append({
            "Name" : name,
            "Team" : team,
            "Position" : position,
            "Attempts" : attempts,
            "Attempts Per Game" : attempts_per_game,
            "Yards" : yards,
            "Average Per Carry" : avg_per_carry,
            "Yards Per Game" : yards_per_game,
            "TDs" : TDs,
            "Twenty Plus": twenty,
            "Forty Plus" : forty,
            "Fumbles" : fum
        })
    time.sleep(.25)
    
    temp_df = pd.DataFrame(rows)
    df_temp_list.append(temp_df)

In [11]:
rb = pd.concat(df_temp_list, ignore_index=True) 
rb.head()

,Attempts,Attempts Per Game,Average Per Carry,Forty Plus,Fumbles,Name,Position,TDs,Team,Twenty Plus,Yards,Yards Per Game
0,303,20.2,5.1,3,5,Derrick Henry,RB,16,TEN,9,1540,102.7
1,298,18.6,5.0,4,2,Nick Chubb,RB,8,CLE,11,1494,93.4
2,242,18.6,4.8,2,1,Josh Jacobs,RB,7,OAK,8,1150,88.5
3,287,17.9,4.8,4,1,Christian McCaffrey,RB,15,CAR,6,1387,86.7
4,301,18.8,4.5,0,3,Ezekiel Elliott,RB,12,DAL,4,1357,84.8


**RB plus Salary**

In [12]:
rb_sal = pd.merge(rb, df, on=["Name", "Position"])
rb_sal.head()

,Attempts,Attempts Per Game,Average Per Carry,Forty Plus,Fumbles,Name,Position,TDs,Team_x,Twenty Plus,Yards,Yards Per Game,Average Guranteed Per Year,Average Per Year,Percent Guranteed,Team_y,Total Guranteed,Total Value
0,303,20.2,5.1,3,5,Derrick Henry,RB,16,TEN,9,1540,102.7,"$10,278,000","$10,278,000",100.0%,Titans,"$10,278,000","$10,278,000"
1,298,18.6,5.0,4,2,Nick Chubb,RB,8,CLE,11,1494,93.4,"$1,324,279","$1,845,774",71.7%,Browns,"$5,297,114","$7,383,094"
2,242,18.6,4.8,2,1,Josh Jacobs,RB,7,OAK,8,1150,88.5,"$2,983,350","$2,983,350",100.0%,Raiders,"$11,933,398","$11,933,398"
3,287,17.9,4.8,4,1,Christian McCaffrey,RB,15,CAR,6,1387,86.7,"$7,515,625","$16,015,853",46.9%,Panthers,"$30,062,500","$64,063,412"
4,301,18.8,4.5,0,3,Ezekiel Elliott,RB,12,DAL,4,1357,84.8,"$4,675,356","$15,000,000",31.2%,Cowboys,"$28,052,137","$90,000,000"


In [13]:
rb_sal.to_csv("NFL_salary_RB_data2020", header=True, index=False)

**WR Data**

In [14]:
df_temp_list = []
for i in [1,2,3,4,5]:
    # i = page number
    resp = requests.get("http://www.nfl.com/stats/categorystats?tabSeq=1&season=2019&seasonType=REG&experience=&Submit=Go&archive=false&d-447263-p={}&conference=null&statisticPositionCategory=WIDE_RECEIVER&qualified=true".format(i))
    soup = BeautifulSoup(resp.content, "html.parser") 

    tables = soup.find_all("table")
    rows = []
    for player in tables[0].find_all("tr")[1:]:
        cell = player.find_all("td")
        name = cell[1].text.strip("\n")
        team = cell[2].text.strip("\n")
        position = cell[3].text
        receptions = int(cell[4].text.strip())
        yards = int(cell[5].text.strip().replace(",", ""))
        if cell[6].text.strip() == "--":
            avg_per_catch = 0.0
        else:
            avg_per_catch = float(cell[6].text.strip())
        yards_per_game = float(cell[7].text.strip())
        TDs = int(cell[9].text.strip())
        twenty = int(cell[10].text.strip())
        forty = int(cell[11].text.strip())
        fum = int(cell[14].text.strip())

        rows.append({
            "Name" : name,
            "Team" : team,
            "Position" : position,
            "Receptions" : receptions,
            "Yards" : yards,
            "Yards Per Catch" : avg_per_catch,
            "Yards Per Game" : yards_per_game,
            "TDs" : TDs,
            "Twenty Plus": twenty,
            "Forty Plus" : forty,
            "Fumbles" : fum
        })
    time.sleep(.25)
    
    temp_df = pd.DataFrame(rows)
    df_temp_list.append(temp_df)

In [15]:
wr = pd.concat(df_temp_list, ignore_index=True) 
wr.head()

,Forty Plus,Fumbles,Name,Position,Receptions,TDs,Team,Twenty Plus,Yards,Yards Per Catch,Yards Per Game
0,3,1,Michael Thomas,WR,149,9,NO,17,1725,11.6,107.8
1,1,0,Keenan Allen,WR,104,6,LAC,15,1199,11.5,74.9
2,1,0,DeAndre Hopkins,WR,104,7,HOU,16,1165,11.2,77.7
3,1,3,Julian Edelman,WR,100,6,NE,13,1117,11.2,69.8
4,3,1,Julio Jones,WR,99,6,ATL,21,1394,14.1,92.9


**WR plus Salary**

In [16]:
wr_sal = pd.merge(wr, df, on=["Name", "Position"])
wr_sal.head()

,Forty Plus,Fumbles,Name,Position,Receptions,TDs,Team_x,Twenty Plus,Yards,Yards Per Catch,Yards Per Game,Average Guranteed Per Year,Average Per Year,Percent Guranteed,Team_y,Total Guranteed,Total Value
0,3,1,Michael Thomas,WR,149,9,NO,17,1725,11.6,107.8,"$7,129,609","$19,250,000",37.0%,Saints,"$35,648,043","$96,250,000"
1,1,0,Keenan Allen,WR,104,6,LAC,15,1199,11.5,74.9,"$5,164,000","$11,250,000",45.9%,Chargers,"$20,656,000","$45,000,000"
2,1,0,DeAndre Hopkins,WR,104,7,HOU,16,1165,11.2,77.7,"$7,300,000","$16,200,000",45.1%,Cardinals,"$36,500,000","$81,000,000"
3,1,3,Julian Edelman,WR,100,6,NE,13,1117,11.2,69.8,"$6,000,000","$7,750,000",77.4%,Patriots,"$12,000,000","$15,500,000"
4,3,1,Julio Jones,WR,99,6,ATL,21,1394,14.1,92.9,"$21,333,333","$22,000,000",97.0%,Falcons,"$64,000,000","$66,000,000"


In [17]:
wr_sal.to_csv("NFL_salary_WR_data2020", header=True, index=False)

**TE Data**

In [18]:
df_temp_list = []
for i in [1,2,3]:
    # i = page number
    resp = requests.get("http://www.nfl.com/stats/categorystats?tabSeq=1&season=2019&seasonType=REG&experience=&Submit=Go&archive=false&d-447263-p={}&conference=null&statisticPositionCategory=TIGHT_END&qualified=true".format(i))
    soup = BeautifulSoup(resp.content, "html.parser") 

    tables = soup.find_all("table")
    rows = []
    for player in tables[0].find_all("tr")[1:]:
        cell = player.find_all("td")
        name = cell[1].text.strip("\n")
        team = cell[2].text.strip("\n")
        position = cell[3].text
        receptions = int(cell[4].text.strip())
        yards = int(cell[5].text.strip().replace(",", ""))
        if cell[6].text.strip() == "--":
            avg_per_catch = 0.0
        else:
            avg_per_catch = float(cell[6].text.strip())
        yards_per_game = float(cell[7].text.strip())
        TDs = int(cell[9].text.strip())
        twenty = int(cell[10].text.strip())
        forty = int(cell[11].text.strip())
        fum = int(cell[14].text.strip())

        rows.append({
            "Name" : name,
            "Team" : team,
            "Position" : position,
            "Receptions" : receptions,
            "Yards" : yards,
            "Yards Per Catch" : avg_per_catch,
            "Yards Per Game" : yards_per_game,
            "TDs" : TDs,
            "Twenty Plus": twenty,
            "Forty Plus" : forty,
            "Fumbles" : fum
        })
    time.sleep(.25)
    
    temp_df = pd.DataFrame(rows)
    df_temp_list.append(temp_df)

In [19]:
te = pd.concat(df_temp_list, ignore_index=True) 
te.head()

,Forty Plus,Fumbles,Name,Position,Receptions,TDs,Team,Twenty Plus,Yards,Yards Per Catch,Yards Per Game
0,3,1,Travis Kelce,TE,97,5,KC,13,1229,12.7,76.8
1,2,1,Darren Waller,TE,90,3,OAK,14,1145,12.7,71.6
2,0,1,Zach Ertz,TE,88,6,PHI,12,916,10.4,61.1
3,2,1,George Kittle,TE,85,5,SF,16,1053,12.4,75.2
4,0,0,Austin Hooper,TE,75,6,ATL,8,787,10.5,60.5


**TE plus Salary**

In [20]:
te_sal = pd.merge(te, df, on=["Name", "Position"])
te_sal.head()

,Forty Plus,Fumbles,Name,Position,Receptions,TDs,Team_x,Twenty Plus,Yards,Yards Per Catch,Yards Per Game,Average Guranteed Per Year,Average Per Year,Percent Guranteed,Team_y,Total Guranteed,Total Value
0,3,1,Travis Kelce,TE,97,5,KC,13,1229,12.7,76.8,"$2,103,400","$9,368,400",22.5%,Chiefs,"$10,517,000","$46,842,000"
1,2,1,Darren Waller,TE,90,3,OAK,14,1145,12.7,71.6,"$805,001","$7,450,001",10.8%,Raiders,"$3,220,004","$29,800,004"
2,0,1,Zach Ertz,TE,88,6,PHI,12,916,10.4,61.1,"$2,663,705","$8,500,000",31.3%,Eagles,"$13,318,527","$42,500,000"
3,2,1,George Kittle,TE,85,5,SF,16,1053,12.4,75.2,"$74,574","$674,574",11.1%,49ers,"$298,296","$2,698,296"
4,0,0,Austin Hooper,TE,75,6,ATL,8,787,10.5,60.5,"$4,625,000","$10,500,000",44.0%,Browns,"$18,500,000","$42,000,000"


In [21]:
te_sal.to_csv("NFL_salary_TE_data2020", header=True, index=False)

**DL Data**

In [22]:
df_temp_list = []
for i in [1,2,3,4,5,6,7]:
    # i = page number
    resp = requests.get("http://www.nfl.com/stats/categorystats?tabSeq=1&season=2019&seasonType=REG&experience=&Submit=Go&archive=false&d-447263-p={}&conference=null&statisticPositionCategory=DEFENSIVE_LINEMAN&qualified=true".format(i))
    soup = BeautifulSoup(resp.content, "html.parser") 

    tables = soup.find_all("table")
    rows = []
    for player in tables[0].find_all("tr")[2:]:
        cell = player.find_all("td")
        name = cell[1].text.strip("\n")
        team = cell[2].text.strip("\n")
        position = cell[3].text
        if cell[5].text.strip() == "--":
            tackles = 0
        else:
            tackles = int(cell[5].text.strip())
        if cell[6].text.strip() == "--":
            assist = 0
        else:
            assist = int(cell[6].text.strip())
        if cell[7].text.strip() == "--":
            sacks = 0.0
        else:
            sacks = float(cell[7].text.strip())
        if cell[8].text.strip() == "--":
            safety = 0
        else:
            safety = int(cell[8].text.strip())
        if cell[9].text.strip() == "--":
            pass_def = 0
        else:
            pass_def = int(cell[9].text.strip())
        if cell[10].text.strip() == "--":
            ints = 0
        else:
            ints = int(cell[10].text.strip())
        if cell[11].text.strip() == "--":
            int_tds = 0
        else:
            int_tds = int(cell[11].text.strip())
        if cell[14].text.strip() == "--":
            forced_fum = 0
        else:
            forced_fum = int(cell[14].text.strip())
        if cell[15].text.strip() == "--":
            fum_rec = 0
        else:
            fum_rec = int(cell[15].text.strip())
        if cell[16].text.strip() == "--":
            fum_tds = 0
        else:
            fum_tds = int(cell[16].text.strip())

        rows.append({
            "Name" : name,
            "Team" : team,
            "Position" : position,
            "Tackles" : tackles,
            "Assists" : assist,
            "Sacks" : sacks,
            "Safetys" : safety,
            "INTs" : ints,
            "INTs TDs": int_tds,
            "Forced Fumbles" : forced_fum,
            "Fumbles Recovered" : fum_rec,
            "Fumbles TDs" : fum_tds
        })
    time.sleep(.25)
    
    temp_df = pd.DataFrame(rows)
    df_temp_list.append(temp_df)

In [23]:
dl = pd.concat(df_temp_list, ignore_index=True) 
dl.head()

,Assists,Forced Fumbles,Fumbles Recovered,Fumbles TDs,INTs,INTs TDs,Name,Position,Sacks,Safetys,Tackles,Team
0,32,1,1,0,0,0,Cameron Heyward,DE,9.0,0,51,PIT
1,30,1,0,0,0,0,Sam Hubbard,DE,8.5,0,46,CIN
2,42,0,0,0,0,0,Davon Godchaux,DT,2.0,0,33,MIA
3,18,3,1,0,0,0,Danielle Hunter,DE,14.5,0,52,MIN
4,31,2,0,0,0,0,Grady Jarrett,DT,7.5,0,38,ATL


**DL plus Salary**

In [24]:
dl_sal = pd.merge(dl, df, on=["Name"])
dl_sal.head()

,Assists,Forced Fumbles,Fumbles Recovered,Fumbles TDs,INTs,INTs TDs,Name,Position_x,Sacks,Safetys,Tackles,Team_x,Average Guranteed Per Year,Average Per Year,Percent Guranteed,Position_y,Team_y,Total Guranteed,Total Value
0,32,1,1,0,0,0,Cameron Heyward,DE,9.0,0,51,PIT,"$3,200,000","$10,456,200",30.6%,34DE,Steelers,"$16,000,000","$52,281,000"
1,30,1,0,0,0,0,Sam Hubbard,DE,8.5,0,46,CIN,"$232,300","$904,800",25.7%,43DE,Bengals,"$929,200","$3,619,200"
2,42,0,0,0,0,0,Davon Godchaux,DT,2.0,0,33,MIA,"$50,487","$650,487",7.8%,34DT,Dolphins,"$201,948","$2,601,948"
3,18,3,1,0,0,0,Danielle Hunter,DE,14.5,0,52,MIN,"$3,381,400","$14,400,000",23.5%,43DE,Vikings,"$16,907,000","$72,000,000"
4,31,2,0,0,0,0,Grady Jarrett,DT,7.5,0,38,ATL,"$9,500,000","$17,000,000",55.9%,43DT,Falcons,"$38,000,000","$68,000,000"


In [25]:
dl_sal.to_csv("NFL_salary_DL_data2020", header=True, index=False)

**LB Data**

In [26]:
df_temp_list = []
for i in [1,2,3,4,5,6]:
    # i = page number
    resp = requests.get("http://www.nfl.com/stats/categorystats?tabSeq=1&season=2019&seasonType=REG&experience=&Submit=Go&archive=false&d-447263-p={}&conference=null&statisticPositionCategory=LINEBACKER&qualified=true".format(i))
    soup = BeautifulSoup(resp.content, "html.parser") 

    tables = soup.find_all("table")
    rows = []
    for player in tables[0].find_all("tr")[2:]:
        cell = player.find_all("td")
        name = cell[1].text.strip("\n")
        team = cell[2].text.strip("\n")
        position = cell[3].text
        if cell[5].text.strip() == "--":
            tackles = 0
        else:
            tackles = int(cell[5].text.strip())
        if cell[6].text.strip() == "--":
            assist = 0
        else:
            assist = int(cell[6].text.strip())
        if cell[7].text.strip() == "--":
            sacks = 0.0
        else:
            sacks = float(cell[7].text.strip())
        if cell[8].text.strip() == "--":
            safety = 0
        else:
            safety = int(cell[8].text.strip())
        if cell[9].text.strip() == "--":
            pass_def = 0
        else:
            pass_def = int(cell[9].text.strip())
        if cell[10].text.strip() == "--":
            ints = 0
        else:
            ints = int(cell[10].text.strip())
        if cell[11].text.strip() == "--":
            int_tds = 0
        else:
            int_tds = int(cell[11].text.strip())
        if cell[14].text.strip() == "--":
            forced_fum = 0
        else:
            forced_fum = int(cell[14].text.strip())
        if cell[15].text.strip() == "--":
            fum_rec = 0
        else:
            fum_rec = int(cell[15].text.strip())
        if cell[16].text.strip() == "--":
            fum_tds = 0
        else:
            fum_tds = int(cell[16].text.strip())

        rows.append({
            "Name" : name,
            "Team" : team,
            "Position" : position,
            "Tackles" : tackles,
            "Assists" : assist,
            "Sacks" : sacks,
            "Safetys" : safety,
            "INTs" : ints,
            "INTs TDs": int_tds,
            "Forced Fumbles" : forced_fum,
            "Fumbles Recovered" : fum_rec,
            "Fumbles TDs" : fum_tds
        })
    time.sleep(.25)
    
    temp_df = pd.DataFrame(rows)
    df_temp_list.append(temp_df)

In [27]:
lb = pd.concat(df_temp_list, ignore_index=True) 
lb.head()

,Assists,Forced Fumbles,Fumbles Recovered,Fumbles TDs,INTs,INTs TDs,Name,Position,Sacks,Safetys,Tackles,Team
0,73,1,1,0,1,0,Bobby Wagner,MLB,3.0,0,86,SEA
1,58,1,0,0,1,0,Blake Martinez,ILB,3.0,0,97,GB
2,57,2,1,0,3,0,Jordan Hicks,MLB,1.5,0,93,ARI
3,63,0,0,0,2,0,Luke Kuechly,MLB,0.0,1,81,CAR
4,43,0,2,0,0,0,Zach Cunningham,ILB,2.0,0,99,HOU


**LB plus Salary**

In [28]:
lb_sal = pd.merge(lb, df, on=["Name"])
lb_sal.head()

,Assists,Forced Fumbles,Fumbles Recovered,Fumbles TDs,INTs,INTs TDs,Name,Position_x,Sacks,Safetys,Tackles,Team_x,Average Guranteed Per Year,Average Per Year,Percent Guranteed,Position_y,Team_y,Total Guranteed,Total Value
0,73,1,1,0,1,0,Bobby Wagner,MLB,3.0,0,86,SEA,"$8,166,667","$18,000,000",45.4%,ILB,Seahawks,"$24,500,000","$54,000,000"
1,58,1,0,0,1,0,Blake Martinez,ILB,3.0,0,97,GB,"$6,333,333","$10,250,000",61.8%,ILB,Giants,"$19,000,000","$30,750,000"
2,57,2,1,0,3,0,Jordan Hicks,MLB,1.5,0,93,ARI,"$4,250,000","$8,500,000",50.0%,ILB,Cardinals,"$17,000,000","$34,000,000"
3,63,0,0,0,2,0,Luke Kuechly,MLB,0.0,1,81,CAR,"$5,400,000","$12,359,059",43.7%,ILB,Panthers,"$27,000,000","$61,795,295"
4,43,0,2,0,0,0,Zach Cunningham,ILB,2.0,0,99,HOU,"$631,358","$1,117,935",56.5%,ILB,Texans,"$2,525,433","$4,471,738"


In [29]:
lb_sal.to_csv("NFL_salary_LB_data2020", header=True, index=False)

**DB Data**

In [30]:
df_temp_list = []
for i in [1,2,3,4,5,6,7,8,9]:
    # i = page number
    resp = requests.get("http://www.nfl.com/stats/categorystats?tabSeq=1&season=2019&seasonType=REG&experience=&Submit=Go&archive=false&d-447263-p={}&conference=null&statisticPositionCategory=DEFENSIVE_BACK&qualified=true".format(i))
    soup = BeautifulSoup(resp.content, "html.parser") 

    tables = soup.find_all("table")
    rows = []
    for player in tables[0].find_all("tr")[2:]:
        cell = player.find_all("td")
        name = cell[1].text.strip("\n")
        team = cell[2].text.strip("\n")
        position = cell[3].text
        if cell[5].text.strip() == "--":
            tackles = 0
        else:
            tackles = int(cell[5].text.strip())
        if cell[6].text.strip() == "--":
            assist = 0
        else:
            assist = int(cell[6].text.strip())
        if cell[7].text.strip() == "--":
            sacks = 0.0
        else:
            sacks = float(cell[7].text.strip())
        if cell[8].text.strip() == "--":
            safety = 0
        else:
            safety = int(cell[8].text.strip())
        if cell[9].text.strip() == "--":
            pass_def = 0
        else:
            pass_def = int(cell[9].text.strip())
        if cell[10].text.strip() == "--":
            ints = 0
        else:
            ints = int(cell[10].text.strip())
        if cell[11].text.strip() == "--":
            int_tds = 0
        else:
            int_tds = int(cell[11].text.strip())
        if cell[14].text.strip() == "--":
            forced_fum = 0
        else:
            forced_fum = int(cell[14].text.strip())
        if cell[15].text.strip() == "--":
            fum_rec = 0
        else:
            fum_rec = int(cell[15].text.strip())
        if cell[16].text.strip() == "--":
            fum_tds = 0
        else:
            fum_tds = int(cell[16].text.strip())

        rows.append({
            "Name" : name,
            "Team" : team,
            "Position" : position,
            "Tackles" : tackles,
            "Assists" : assist,
            "Sacks" : sacks,
            "Safetys" : safety,
            "INTs" : ints,
            "INTs TDs": int_tds,
            "Forced Fumbles" : forced_fum,
            "Fumbles Recovered" : fum_rec,
            "Fumbles TDs" : fum_tds
        })
    time.sleep(.25)
    
    temp_df = pd.DataFrame(rows)
    df_temp_list.append(temp_df)

In [31]:
db = pd.concat(df_temp_list, ignore_index=True) 
db.head()

,Assists,Forced Fumbles,Fumbles Recovered,Fumbles TDs,INTs,INTs TDs,Name,Position,Sacks,Safetys,Tackles,Team
0,43,1,1,0,0,0,Budda Baker,SS,0.5,0,104,ARI
1,33,1,2,0,0,0,Eric Reid,SS,4.0,0,97,CAR
2,39,2,0,0,0,0,Landon Collins,SS,1.0,0,78,WAS
3,34,0,0,0,1,0,Shawn Williams,SS,1.0,0,80,CIN
4,40,4,0,0,4,0,Logan Ryan,CB,4.5,0,73,TEN


**DB plus Salary**

In [32]:
db_sal = pd.merge(db, df, on=["Name"])
db_sal.head()

,Assists,Forced Fumbles,Fumbles Recovered,Fumbles TDs,INTs,INTs TDs,Name,Position_x,Sacks,Safetys,Tackles,Team_x,Average Guranteed Per Year,Average Per Year,Percent Guranteed,Position_y,Team_y,Total Guranteed,Total Value
0,43,1,1,0,0,0,Budda Baker,SS,0.5,0,104,ARI,"$1,118,176","$1,707,493",65.5%,S,Cardinals,"$4,472,705","$6,829,970"
1,39,2,0,0,0,0,Landon Collins,SS,1.0,0,78,WAS,"$5,166,667","$14,000,000",36.9%,S,Redskins,"$31,000,000","$84,000,000"
2,34,0,0,0,1,0,Shawn Williams,SS,1.0,0,80,CIN,"$1,000,000","$4,875,000",20.5%,S,Bengals,"$4,000,000","$19,500,000"
3,40,4,0,0,4,0,Logan Ryan,CB,4.5,0,73,TEN,"$4,000,000","$10,000,000",40.0%,CB,Titans,"$12,000,000","$30,000,000"
4,36,3,3,0,2,0,Jordan Poyer,FS,1.0,0,71,BUF,"$4,262,500","$9,750,000",43.7%,S,Bills,"$8,525,000","$19,500,000"


In [33]:
db_sal.to_csv("NFL_salary_DB_data2020", header=True, index=False)